In [34]:
import pandas as pd
from time import sleep
from slugify import slugify
import random
import re
import numpy as np

from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

from selenium_stealth import stealth

pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [58]:
def generateDriver():
    options = webdriver.ChromeOptions() 
    options.add_argument("user-data-dir=selenium")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage') 
    options.add_argument("start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled") 
    options.add_experimental_option('excludeSwitches', ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options = options)
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine")

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
    driver.get('https://www.liveauctioneers.com/search/')
    return driver

driver = generateDriver()

In [43]:
listings = pd.read_csv('data/listings/listings_all.csv')
urls = listings.auctioneer_url.unique()

els = []
i = 0
lats = []
lngs = []
cities = []
states = []
addresses = []
zips = []
auctions = []

def scrape():
    global i
    global driver
    
    for url in urls[i:]:
        foundJSON = False
        try:
            driver.get(url)
        except:
            print('crashed but restarting')
            driver = generateDriver()
            sleep(10)
            scrape()
        
        print(url)
        
        if driver.find_elements(By.XPATH,"//*[contains(text(),'PostalAddress')]"):
            json = driver.find_elements(By.XPATH,"//*[contains(text(),'PostalAddress')]")[0].get_attribute('innerHTML')
            state = re.search(r"\"addressRegion\":.*", json).group().replace('\"addressRegion\": ', '').replace(',', '').replace('"', '')
            address = re.search(r"\"streetAddress\":.*", json).group().replace('\"streetAddress\": ', '').replace(',', '').replace('"', '')
            city = re.search(r"\"addressLocality\":.*", json).group().replace('\"addressLocality\": ', '').replace(',', '').replace('"', '')
            zip_code = re.search(r"\"postalCode\":.*", json).group().replace('\"postalCode\": ', '').replace(',', '').replace('"', '')
            auctionz = int(driver.find_element(By.CSS_SELECTOR, '[aria-controls="react-autowhatever-search-input"]').get_attribute('placeholder').split(' ')[0].replace(',',''))
        else:
            lat = lng = city = state = address = zip_code = auctionz = np.nan

        if driver.find_elements(By.XPATH, "//*[contains(text(),'\"latitude\": ')]"):
            latJSON = driver.find_elements(By.XPATH,"//*[contains(text(),'\"latitude\": ')]")[0].get_attribute('innerHTML')
            lat = float(re.search(r"\"latitude\":.*", latJSON).group().replace('\"latitude\": ', '').replace(',', '').replace('"', ''))
            lng = float(re.search(r"\"longitude\":.*", latJSON).group().replace('\"longitude\": ', '').replace(',', '').replace('"', ''))
        else:
            lat = np.nan
            lng = np.nan

        lats.append(lat)
        lngs.append(lng)
        cities.append(city)
        states.append(state)
        addresses.append(address)
        zips.append(zip_code)
        auctions.append(auctionz)

        i += 1
        print(i)
        
    print('all done')
    
scrape()

https://www.liveauctioneers.com/auctioneer/897/harrison-auctions-inc/


AttributeError: 'NoneType' object has no attribute 'group'

In [51]:
df = pd.DataFrame(list(zip(urls, lats, lngs, states, cities, zips, addresses, auctions)),
                  columns=['auctioneer_url', 'lat', 'lng', 'state', 'city', 'zip', 'address', 'auctions'])
df.to_csv('data/auctioneers.csv', index = False)
len(df)
df.to_clipboard()
# df1 = pd.read_csv('data/auctioneers.csv')
# df2 = pd.DataFrame(list(zip(urls, lats, lngs, states, cities, zips, addresses, auctions)),
                  columns=['auctioneer_url', 'lat', 'lng', 'state', 'city', 'zip', 'address', 'auctions'])
# df2
# df3 = pd.concat([df1, df2]).drop_duplicates().reset_index(drop = True)
# df3.to_csv('data/auctioneers.csv', index = False)
# len(df3)

,auctioneer_url,lat,lng,state,city,zip,address,auctions
0,https://www.liveauctioneers.com/auctioneer/897/harrison-auctions-inc/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.liveauctioneers.com/auctioneer/899/gulfcoast-coin-and-jewelry/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.liveauctioneers.com/auctioneer/1011/universal-live/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.liveauctioneers.com/auctioneer/21/austin-auction-gallery/,30.457,-97.7658,NaN,NaN,NaN,NaN,NaN
4,https://www.liveauctioneers.com/auctioneer/385/william-bunch-auctions-and-appraisals/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://www.liveauctioneers.com/auctioneer/686/midwest-auction-galleries-inc/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://www.liveauctioneers.com/auctioneer/610/nette-auctions/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.liveauctioneers.com/auctioneer/135/signature-house/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,https://www.liveauctioneers.com/auctioneer/179/quinn-s-auction-galleries/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,https://www.liveauctioneers.com/auctioneer/170/auctions-neapolitan/,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# Fill in missing lat lngs that were manually geocoded
df1 = pd.read_csv('data/auctioneers/auctioneers.csv')
df2 = pd.read_csv('data/auctioneers/auctioneers_missing_latlng.csv')
df3 = pd.concat([df1, df2]).drop_duplicates().reset_index(drop = True)
df4 = df3.loc[df3.notnull().sum(1).groupby([df3.auctioneer_url, df3.lat]).idxmax()]
df4.to_csv('data/auctioneers/auctioneers_all.csv', index = False)
# df4[df4['lat'].isna()]

In [78]:
# Join with listings
df1 = pd.read_csv('data/listings/listings_all.csv')
df2 = pd.read_csv('data/auctioneers/auctioneers_all.csv')
df3 = pd.merge(df1, df2, how="left", on=['auctioneer_url'])
df3['price'] = df3.price.str.replace(',', '').str.replace('$', '').astype('Int32')
df3['date'] = pd.to_datetime(df3.date, format = '%b %d, %Y')
df3['cats'] = df3.cats.str.replace('&amp;', '&')
df3.to_csv('data/listings/listings_merged.csv', index = False)